In [1]:
#!/usr/bin/env python
# from types import TupleType
'''
This module implement a typical K-means Clustering. The original author is Jing Wang (hbhzwj@gmail.com);
cf. https://github.com/hbhzwj/GAD/blob/eadf9fe5c8749bb7c091c41a6ab0e8488a7f8619/gad/Detector/ClusterAlg.py
'''
from __future__ import print_function, division, absolute_import
from random import sample

try:
    import Queue as queue # replace with 'import queue' if using Python 3
except ImportError:
    import queue
    
## {{{ http://code.activestate.com/recipes/577661/ (r2)
# from sadit.util import queue
class MedianHeap:
    def __init__(self, numbers = None):
        self.median = None
        self.left = queue.PriorityQueue() # max-heap
        self.right = queue.PriorityQueue() # min-heap
        self.diff = 0 # difference in size between left and right

        if numbers:
            for n in numbers:
                self.put(n)

    def top(self):
        return self.median

    def put(self, n):
        if not self.median:
            self.median = n
        elif n <= self.median:
            self.left.put(-n)
            self.diff += 1
        else:
            self.right.put(n)
            self.diff -= 1

        if self.diff > 1:
            self.right.put(self.median)
            self.median = -self.left.get()
            self.diff = 0
        elif self.diff < -1:
            self.left.put(-self.median)
            self.median = self.right.get()
            self.diff = 0

    def get(self):
        median = self.median

        if self.diff > 0:
            self.median = -self.left.get()
            self.diff -= 1
        elif self.diff < 0:
            self.median = self.right.get()
            self.diff += 1
        elif not self.left.empty():
            self.median = -self.left.get()
            self.diff -= 1
        elif not self.right.empty():
            self.median = self.right.get()
            self.diff += 1
        else: # median was the only element
            self.median = None

        return median

In [2]:
try:
    import numpy as np
    NUMPY = True
except:
    NUMPY = False

def GetMedian(numbers):
    if NUMPY:
        return float(np.median(numbers))
    else:
        m = MedianHeap(numbers)
        return m.get()

def ReClustering(data, centerPt, DF):
    cluster = []
    for pt in data:
        dv = []
        for c in centerPt:
            d = DF(pt, c)
            dv.append(d)
        md = min(dv)
        cluster.append(dv.index(md))
    return cluster

def Add(x, y):
    res = []
    for i in xrange(len(x)):
        res.append(x[i]+y[i])

    return res

def NDivide(cSum, cLen):
    res = []
    for i in xrange(len(cSum)):
        x = []
        for j in xrange(len(cSum[0])):
            x.append( cSum[i][j] * 1.0 / cLen[i] )

        res.append(x)
    return res

def CalClusteringCenterKMeans(data, cluster):
    ucLen = max(cluster) + 1
    cSum = [ [0] * len(data[0]) for i in xrange(ucLen)]
    cLen = [0] * ucLen
    i = -1
    for ele in data:
        i += 1
        cl = cluster[i]
        cSum[cl] = Add(cSum[cl], ele)
        cLen[cl] += 1

    try:
        cAve = NDivide(cSum, cLen)
    except:
        import pdb; pdb.set_trace()
    return cAve

def CalClusteringCenterKMedians(data, cluster):
    ucLen = max(cluster) + 1
    cMed = []
    for cl in xrange(ucLen):
        clusterData = [d for i, d in zip(cluster, data) if i == cl]
        med = [GetMedian(vals) for vals in zip(*clusterData)]
        cMed.append(med)
    return cMed

def Equal(x, y):
    if len(x) != len(y):
        return False
    for i in range(len(x)):
        if x[i] != y[i]:
            return False
    return True

def KMeans(data, k, distFunc):
# def KMedians(data, k, distFunc):
    print('start KMeans...')
    # Initialize
    centerPt = sample(data, k)
    while True:
        cluster = ReClustering(data, centerPt, distFunc)
        NewCenterPt = CalClusteringCenterKMeans(data, cluster)
        if Equal(NewCenterPt, centerPt):
            break
        centerPt = NewCenterPt

    return cluster, centerPt

def KMedians(data, k, distFunc):
# def KMeans(data, k, distFunc):
    print('start KMedians ...')
    # Initialize
    centerPt = sample(data, k)
    while True:
        cluster = ReClustering(data, centerPt, distFunc)
        NewCenterPt = CalClusteringCenterKMedians(data, cluster)
        if Equal(NewCenterPt, centerPt):
            break
        centerPt = NewCenterPt

    return cluster, centerPt

In [23]:
##### Define the distance between Jam A and Jam B

from geopy.distance import vincenty

def DF(A_, B_):
    '''
    The argument A_ and B_ should be two lists
    '''
    A_ = np.array(A_)
    B_ = np.array(B_)
    A = np.reshape(A_, (int(np.size(A_) / 2), 2))
    B = np.reshape(B_, (int(np.size(B_) / 2), 2))
    len_A = np.size(A, 0)
    len_B = np.size(B, 0)

    dist_AB_ = []
    for i in range(len_A):
        for j in range(len_B):
            jam1 = (A[i,1], A[i,0])
            jam2 = (B[j,1], B[j,0])
            dist_AB_.append(vincenty(jam1, jam2).meters)
    dist_AB = min(dist_AB_)
    return dist_AB

In [24]:
A = np.array([[-71.09702, 42.344783],
  [-71.095651, 42.345145],
  [-71.095584, 42.345162],
  [-71.095326, 42.345231],
  [-71.095052, 42.345308],
  [-71.094946, 42.345339],
  [-71.094644, 42.345437],
  [-71.094461, 42.345497],
  [-71.094333, 42.345564],
  [-71.09428, 42.345602],
  [-71.093212, 42.346572],
  [-71.09309, 42.346661],
  [-71.092813, 42.346817]])

B = np.array([[-71.093802, 42.367956],
              [-71.093802, 42.367956],
              [-71.093802, 42.367956],
              [-71.093802, 42.367956],
              [-71.093802, 42.367956],
              [-71.093802, 42.367956],
              [-71.093802, 42.367956],
              [-71.093802, 42.367956],
              [-71.093802, 42.367956],
              [-71.093802, 42.367956],
  [-71.093229, 42.367484],
  [-71.092506, 42.366904],
  [-71.091803, 42.366315]])

A_ = list(np.reshape(A,26))
B_ = list(np.reshape(B,26))

In [25]:
data = [A_, A_, B_]

In [26]:
data

[[-71.097020000000001,
  42.344783,
  -71.095651000000004,
  42.345145000000002,
  -71.095584000000002,
  42.345162000000002,
  -71.095326,
  42.345230999999998,
  -71.095051999999995,
  42.345308000000003,
  -71.094945999999993,
  42.345339000000003,
  -71.094644000000002,
  42.345436999999997,
  -71.094460999999995,
  42.345497000000002,
  -71.094333000000006,
  42.345564000000003,
  -71.094279999999998,
  42.345602,
  -71.093211999999994,
  42.346572000000002,
  -71.093090000000004,
  42.346660999999997,
  -71.092813000000007,
  42.346817000000001],
 [-71.097020000000001,
  42.344783,
  -71.095651000000004,
  42.345145000000002,
  -71.095584000000002,
  42.345162000000002,
  -71.095326,
  42.345230999999998,
  -71.095051999999995,
  42.345308000000003,
  -71.094945999999993,
  42.345339000000003,
  -71.094644000000002,
  42.345436999999997,
  -71.094460999999995,
  42.345497000000002,
  -71.094333000000006,
  42.345564000000003,
  -71.094279999999998,
  42.345602,
  -71.093211999999

In [45]:
def test():
#     data = [(5, 6, 8, 9),
#             (10, 0, 4, 1),
#             (10, 0, 2, 1),
#             (10, 0, 5, 1),
#             (10, 20, 30, 1),
#             (10, 20, 30, 2),
#             (10, 20, 30, 3),
#             (10, 20, 30, 4),
#             (1, 1, 1, 1),
#             (10, 20, 30, 6),
#             (10, 20, 30, 7),
#             (10, 20, 30, 8),
#             (10, 200, 1, 1),
#             (10, 20, 30, 9),
#             (10, 0, 3, 1),
#             (2, 3, 5, 6),
#             (10, 20, 30, 5)
#             ]
    data = [A_, A_, B_, B_, B_, A_, B_, A_, B_, B_, A_, A_]
    print(data)
    k = 5
    
#     DF = lambda x,y:abs(x[0]-y[0]) * (256**3) + abs(x[1]-y[1]) * (256 **2) + abs(x[2]-y[2]) * 256 + abs(x[3]-y[3])
    # DF = lambda x,y:abs(x[0]-y[0]) * 2 + abs(x[1]-y[1]) * 2 + abs(x[2]-y[2]) * 2+ abs(x[3]-y[3]) * 2
    # DF = lambda x,y: ((x[0]-y[0]) ** 2) * (256 ** 3) + ((x[1]-y[1]) ** 2) * (256 **2) + ((x[2]-y[2]) ** 2) * (256)+ (x[3]-y[3]) ** 2
    # DF = lambda x,y:(x[0]-y[0]) ** 2  + (x[1]-y[1]) ** 2  + (x[2]-y[2]) ** 2 + (x[3]-y[3]) ** 2

    cluster, centerPt = KMedians(data, k, DF)
    print('cluster, ', cluster)
    print('centerPt, ', centerPt)

# if __name__ == "__main__":
#     from time import clock
#     tic = clock()
#     for i in xrange(1000):
#         test()
#     toc = clock()
#     print('total simulation time is %f'%(toc-tic))
#     # m = MedianHeap([1.4, 2.1, 5.0, 5.0])
#     # print('median, ', m.get())

In [46]:
test()

[[-71.097020000000001, 42.344783, -71.095651000000004, 42.345145000000002, -71.095584000000002, 42.345162000000002, -71.095326, 42.345230999999998, -71.095051999999995, 42.345308000000003, -71.094945999999993, 42.345339000000003, -71.094644000000002, 42.345436999999997, -71.094460999999995, 42.345497000000002, -71.094333000000006, 42.345564000000003, -71.094279999999998, 42.345602, -71.093211999999994, 42.346572000000002, -71.093090000000004, 42.346660999999997, -71.092813000000007, 42.346817000000001], [-71.097020000000001, 42.344783, -71.095651000000004, 42.345145000000002, -71.095584000000002, 42.345162000000002, -71.095326, 42.345230999999998, -71.095051999999995, 42.345308000000003, -71.094945999999993, 42.345339000000003, -71.094644000000002, 42.345436999999997, -71.094460999999995, 42.345497000000002, -71.094333000000006, 42.345564000000003, -71.094279999999998, 42.345602, -71.093211999999994, 42.346572000000002, -71.093090000000004, 42.346660999999997, -71.092813000000007, 42.3

In [8]:
data = [(5, 6, 8, 9),
        (10, 0, 4, 1),
        (10, 0, 2, 1),
        (10, 0, 5, 1),
        (10, 20, 30, 1),
        (10, 20, 30, 2),
        (10, 20, 30, 3),
        (10, 20, 30, 4),
        (1, 1, 1, 1),
        (10, 20, 30, 6),
        (10, 20, 30, 7),
        (10, 20, 30, 8),
        (10, 200, 1, 1),
        (10, 20, 30, 9),
        (10, 0, 3, 1),
        (2, 3, 5, 6),
        (10, 20, 30, 5)
        ]

In [9]:
data

[(5, 6, 8, 9),
 (10, 0, 4, 1),
 (10, 0, 2, 1),
 (10, 0, 5, 1),
 (10, 20, 30, 1),
 (10, 20, 30, 2),
 (10, 20, 30, 3),
 (10, 20, 30, 4),
 (1, 1, 1, 1),
 (10, 20, 30, 6),
 (10, 20, 30, 7),
 (10, 20, 30, 8),
 (10, 200, 1, 1),
 (10, 20, 30, 9),
 (10, 0, 3, 1),
 (2, 3, 5, 6),
 (10, 20, 30, 5)]